In [31]:
#Import packages needed
import pandas as pd
from sqlalchemy import create_engine
import requests
import gmaps
from config import gkey
from pprint import pprint
gmaps.configure(api_key=gkey)

## Bringing in the data

In [5]:
#Bringing in cime data and making it a dataframe
crimes_csv = "LACrimes2013Zip.csv"
crimes_df = pd.read_csv(crimes_csv)
crimes_df.head()

,Date.Rptd,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Status,Status.Desc,LOCATION,Cross.Street,Location.1,Formatted Address,Zipcode
0,03/20/2013,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,UNK,Unknown,OXFORD,OAKWOOD,"(34.0776, -118.308)","4650-4652 Oakwood Ave, Los Angeles, CA 90004, USA",90004
1,03/10/2013,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,UNK,Unknown,ODIN ST,CAHUENGA BL,"(34.1113, -118.3336)","2314 N Cahuenga Blvd, Los Angeles, CA 90068, USA",90068
2,12/18/2013,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,UNK,Unknown,105TH ST,CROESUS AV,"(33.9406, -118.2338)","2148 E 105th St, Los Angeles, CA 90002, USA",90002
3,10/18/2013,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,UNK,Unknown,101ST ST,JUNIPER ST,"(33.9449, -118.2332)","10100-10198 Juniper St, Los Angeles, CA 90002,...",90002
4,05/26/2013,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),UNK,Unknown,1300 W SEPULVEDA BL,NaN,"(33.8135, -118.2992)","1303 Sepulveda Blvd, Torrance, CA 90501, USA",90501


In [6]:
#Bringing in bike data and making it a dataframe
bike_csv = "los-angeles-metro-bike-share-trip-data/metro-bike-share-trip-data.csv"
bike_df = pd.read_csv(bike_csv)
bike_df.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Trip Route Category,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9518671,77160,2016-10-09T14:37:00.000,2016-10-10T12:03:00.000,NaN,NaN,NaN,3000.0,NaN,NaN,...,One Way,Monthly Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21031476,86400,2017-02-27T20:26:00.000,2017-03-01T09:49:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,One Way,Flex Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Looking at and cleaning the data

In [9]:
# Look at the size of the data
crimes_df.shape

(201764, 16)

In [8]:
# Look at the size of the data
bike_df.shape

(132427, 22)

In [17]:
# Check to see if any NaN's in crimes and how much
crimes_df["Zipcode"].isnull().sum()

0

In [18]:
# Check to see if any NaN's in crimes and how much
bike_df["Zip Codes"].isnull().sum()

33837

In [74]:
# Drop the NaN's in bikes and check to see how much data is left
bike_df_no_nans_zip = bike_df.dropna(subset=["Zip Codes", "Starting Station Latitude", "Starting Station Longitude"])
bike_df_no_nans_zip.shape

(98590, 22)

In [75]:
bike_df_no_nans_zip.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Trip Route Category,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts
14,16883024,11100,2016-12-28T15:27:00.000,2016-12-28T18:32:00.000,3016.0,34.052898,-118.241562,NaN,NaN,NaN,...,One Way,Staff Annual,"{'longitude': '-118.241562', 'latitude': '34.0...",NaN,76.0,9.0,22728.0,NaN,NaN,NaN
32,15177219,13920,2016-12-05T10:43:00.000,2016-12-05T14:35:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,One Way,Walk-up,"{'longitude': '-118.262733', 'latitude': '34.0...",NaN,76.0,9.0,23078.0,NaN,NaN,NaN
35,12227457,14580,2016-11-03T17:21:00.000,2016-11-03T21:24:00.000,3027.0,34.049980,-118.247162,3000.0,NaN,NaN,...,One Way,Flex Pass,"{'longitude': '-118.247162', 'latitude': '34.0...",NaN,76.0,9.0,23074.0,NaN,NaN,NaN
40,16726207,1320,2016-12-25T17:15:00.000,2016-12-25T17:37:00.000,3078.0,34.064281,-118.238937,3000.0,NaN,NaN,...,One Way,Walk-up,"{'longitude': '-118.238937', 'latitude': '34.0...",NaN,28.0,11.0,22728.0,NaN,NaN,NaN
43,13913164,11160,2016-11-19T15:39:00.000,2016-11-19T18:45:00.000,3025.0,34.032860,-118.268082,3000.0,NaN,NaN,...,One Way,Walk-up,"{'longitude': '-118.268082', 'latitude': '34.0...",NaN,76.0,9.0,23076.0,NaN,NaN,NaN


In [24]:
# Get only the columns that we need and make a new dataframe
new_crimes_df = crimes_df[["DR.NO", "Zipcode"]].copy()
new_crimes_df.head()

,DR.NO,Zipcode
0,132007717,90004
1,130608787,90068
2,131820260,90002
3,131817514,90002
4,130510483,90501


In [25]:
new_crimes_df.shape

(201764, 2)

In [26]:
new_bike_df = bike_df_no_nans_zip[["Trip ID", "Duration", "Zip Codes"]].copy()
new_bike_df.head()

,Trip ID,Duration,Zip Codes
14,16883024,11100,22728.0
32,15177219,13920,23078.0
35,12227457,14580,23074.0
40,16726207,1320,22728.0
43,13913164,11160,23076.0


In [27]:
new_bike_df["Zip Codes"].unique()

array([22728., 23078., 23074., 23076., 23075., 23082., 24038., 22724.])

In [76]:
bike_df_no_nans_zip["Starting Station Latitude"].isnull().sum()

0

In [77]:
bike_df_no_nans_zip["Starting Station Longitude"].isnull().sum()

0

In [ ]:
for index, row in bike_df.iterrows():

    # Get the address from dataframe
    address = row['City Address']
    
    # Build the url and gkey
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(address, gkey)


    # assemble url and make API request
    response = requests.get(target_url).json()

In [ ]:
for index, row in bike_df.iterrows():
    
    lat = row["Starting Station Latitude"]
    long = row["Starting Station Longitude"]


    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
              'latlng={lat},{long}&key={gkey}').format(lat=lat, long=long, gkey=gkey)
    response = requests.get(target_url).json()
    
    try:
        print(response['results'][0]['address_components'][7]['long_name'])
    except:
        print(f"Could not {index} find zip")
        
        
    
#     target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
#               'latlng={0,1}&key={2}').format(lat, long, gkey)
    


90014
Could not 1 find zip
90012
90012
United States
Could not 5 find zip
90012
90013
90013
90012
90013
90017
Could not 12 find zip
90013
90012
Could not 15 find zip
90013
90012
Could not 18 find zip
90017
90012
90021
Could not 22 find zip
90017
Could not 24 find zip
90012
90013
90012
90015
90012
90013
90017
United States
90014
90014
90013
90014
90015
90013
90012
90012
90013
90013
90015
90015
90015
90013
90021
90012
90012
90013
Could not 51 find zip
90012
90017
Could not 54 find zip
90013
90012
90012
90015
90012
90013
90012
90014
Could not 63 find zip
Could not 64 find zip
90013
90013
Could not 67 find zip
90015
90012
90013
90012
90071
90015
90015
Could not 75 find zip
90017
90014
90014
90014
90013
90012
90014
90013
90013
90015
Could not 86 find zip
90014
90071
90012
90012
90013
90013
Could not 93 find zip
Could not 94 find zip
United States
90012
90014
90014
90012
90014
90012
90012
90015
90071
90013
90014
Could not 107 find zip
90012
90013
90015
90017
90014
Could not 113 find zip
9001

In [66]:
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
              'latlng=34.052898,-118.241562&key={0}').format(gkey)
response = requests.get(target_url).json()
pprint(response['results'][0]['address_components'][7]['long_name'])

'90012'


In [43]:
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=gkey')
response = requests.get(target_url).json()
print(response)

{'error_message': 'The provided API key is invalid.', 'results': [], 'status': 'REQUEST_DENIED'}
